In [35]:
import matplotlib.pyplot as plt
import math
import matplotlib.patches as patches
import numpy as np
import matplotlib
matplotlib.use('nbagg')
import matplotlib.animation as anm

from IPython.display import HTML # アニメーションが表示されないための処置

In [36]:
class World:
    def __init__(self, debug=False):
        self.objects = [] #ここにロボットなどのオブジェクトを登録
        self.debug   = debug
    
    def append(self, obj): #オブジェクトを登録するための関数
        self.objects.append(obj)
    
    def draw(self):
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_subplot(111)
        ax.set_aspect('equal')
        ax.set_xlim(-5, 5)
        ax.set_ylim(-5, 5)
        ax.set_xlabel("X", fontsize=10)
        ax.set_ylabel("Y", fontsize=10)

        elems = []

        if self.debug:
            for i in range(1000):
                self.one_step(i, elems, ax) #デバッグ時はアニメーション表示させない
        else:
            self.anm = anm.FuncAnimation(fig, self.one_step, fargs=(elems, ax), frames=10, interval=1000, repeat=False)
            plt.show()
            # LiveShareのParticipantsは描画されない
            # アニメーションが自動で再生されない

        plt.close()                         #アニメーションが表示されないための処置
        return HTML(self.anm.to_jshtml())   #アニメーションが表示されないための処置


    def one_step(self, i, elems, ax):
        while elems:
            elems.pop().remove()
        elems.append(ax.text(-4.4, 4.5, "t=" + str(i), fontsize=10))
        for obj in self.objects:
            obj.draw(ax, elems)
            if hasattr(obj, "one_step"): obj.one_step(1.0) #　robotのone_step呼び出し


In [37]:
class IdealRobot:
    def __init__(self,pose,agent=None, color="black"):
        self.pose = pose    # 引数から姿勢の初期値を設定
        self.r = 0.2        # これは描画のためなので固定値（ロボットの丸の大きさ）
        self.color = color  # 引数から描画するときの色を設定
        self.agent = agent
        self.poses = [pose] # 軌跡の描画用

    def draw(self, ax, elems):
        x, y, theta = self.pose     # 姿勢の変数を分解して3つの変数へ
        xn = x + self.r * math.cos(theta)    # ロボットの鼻先のx座標
        yn = y + self.r * math.sin(theta)    # ロボットの鼻先のy座標
        elems += ax.plot([x, xn],[y, yn], color=self.color) # ロボットの向きを示す線分の描画
        c = patches.Circle(xy=(x,y), radius=self.r, fill=False,color=self.color)
        elems.append(ax.add_patch(c)) # 上のpatches.Circleでロボットの胴体を示す円を作ってサブプロットへ登録

        self.poses.append(self.pose)
        elems += ax.plot([e[0] for e in self.poses], [e[1] for e in self.poses], linewidth=0.5, color="black")

    def one_step(self, time_interval):
        if not self.agent: return
        nu, omega = self.agent.decision()
        self.pose = self.state_transition(nu, omega, time_interval, self.pose)

    @classmethod
    def state_transition(cls, nu, omega, time, pose): 
        theta0 = pose[2]
        if math.fabs(omega) < 1e-10: # 角速度がほとんどゼロのとき
            return pose + np.array([nu*math.cos(theta0),nu*math.sin(theta0),omega]) * time
        else:
            return pose + np.array([nu/omega*(math.sin(theta0+omega*time)-math.sin(theta0)), 
                                    nu/omega*(-math.cos(theta0+omega*time)+math.cos(theta0)),
                                    omega*time])

In [38]:
# IdealRobot.state_transition(0.1, 0.0, 1.0, np.array([0,0,0]).T)
# IdealRobot.state_transition(0.1, 10./180.*math.pi, 9.0, np.array([0,0,0]).T)
IdealRobot.state_transition(0.1, 10./180.*math.pi, 18.0, np.array([0,0,0]).T)

array([7.01670930e-17, 1.14591559e+00, 3.14159265e+00])

In [39]:
class Agent:
    def __init__(self, nu, omega):
        self.nu = nu
        self.omega = omega
    
    def decision(self, observation=None):
        return self.nu, self.omega
    
    

In [40]:
world = World()
# world.draw()
straight = Agent(0.2, 0.0)              # 直進する
circling = Agent(0.2, 10.0/180*math.pi) # 円を描く

### 以下追加(3.2.2) ###
robot1 = IdealRobot(np.array([2,3,math.pi/6]).T, straight)          # ロボットのインスタンス生成（色を省略）
robot2 = IdealRobot(np.array([2,-1,math.pi/5*6]).T, circling, "red") # ロボットのインスタンス生成（色を指定）
robot3 = IdealRobot(np.array([0,0,0]).T, color="blue")          # ロボットのインスタンス生成（色を省略）

world.append(robot1)
world.append(robot2)
world.append(robot3)
world.draw()


<IPython.core.display.Javascript object>

In [41]:
a = 100

In [42]:
Mitsuru31 = 10